<a href="https://colab.research.google.com/github/GianmarcoLattaruolo/Vision_Project/blob/main/Classificazione_in_localizzabile_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
import sys
from pathlib import Path
from PIL import Image
from numpy import asarray
import multiprocessing as mp

In [1]:
# with this line we can check if we are in colab or not
in_colab = 'google.colab' in sys.modules
print("are we in Colab?:",in_colab)

cwd = Path(os.getcwd())
if in_colab:
    from google.colab import drive
    drive.mount('/gdrive')
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    os.chdir('/gdrive/MyDrive/GeoEstimation')
    
else:
    #our defult wd in local should be Vision_Project
    if str(cwd)[-14:] == 'Vision_Project':
        os.chdir(cwd / 'GeoEstimation')
    sys.path.append(cwd / 'GeoEstimation')

are we in Colab?: True
Mounted at /gdrive
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:21
🔁 Restarting kernel...


In [5]:
dataset = pd.read_csv("/gdrive/MyDrive/GeoEstimation/resources/im2gps3k_places365_classified.csv").dropna()

In [13]:
images = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]

In [19]:
#images = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]
#img_all = [Image.open(image) for image in images]
#img_data = [asarray(img) for img in img_all]

In [7]:
# a function to load all the images in an array format in img_array
# other "easier solutions" were really slow, this works fine

def load_image(image_path):
    with Image.open(image_path) as img:
        img = img.resize((32, 32))
        return np.asarray(img)

if __name__ == '__main__':
    # Crea una pool di processi
    with mp.Pool() as pool:
        # Carica le immagini in parallelo
        images = ["/gdrive/MyDrive/GeoEstimation/resources/images/im2gps3k/{}".format(img) for img in dataset['IMG_ID']]
        img_data = pool.map(load_image, images)
    
    # Crea un array multidimensionale a partire dalla lista di array numpy
    img_array = np.stack(img_data)


In [9]:
img_array.shape

(1218, 32, 32, 3)

In [17]:
# the labels
label_array = np.array(dataset['LOC']).reshape((1218, 1))

In [18]:
label_array.shape

(1218, 1)

In [32]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(img_array, label_array, test_size=0.16, random_state=123)

In [31]:
# processing data for one-hot encoding
def preprocess_data(X, Y):
  X_p = preprocess_input(X)
  Y_p = to_categorical(Y, 3)
  return X_p, Y_p

In [33]:
# one hot encoding applied both to train and test sets
X_train, y_train = preprocess_data(X_train, y_train)
X_test, y_test = preprocess_data(X_test, y_test)

In [36]:
# the model
input_t = Input(shape=(32, 32, 3))
res_model = ResNet50(include_top=False, #resnet50
                   weights='imagenet', #pretrained from imagenet
                   input_tensor=input_t)

94765736/94765736 [==============================] - 1s 0us/step


In [37]:
# set all the layers not-trainable
for layer in res_model.layers[:]:
  layer.trainable = False

In [38]:
# add trainable layers to performe the classification
model = Sequential()
model.add(res_model)
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

In [40]:
# compile and fit
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=5, verbose=1)
model.summary()

Epoch 1/5
32/32 [==============================] - 11s 220ms/step - loss: 2.3239 - accuracy: 0.6061
Epoch 2/5
32/32 [==============================] - 7s 202ms/step - loss: 2.2093 - accuracy: 0.6334
Epoch 3/5
32/32 [==============================] - 5s 143ms/step - loss: 1.4615 - accuracy: 0.7038
Epoch 4/5
32/32 [==============================] - 6s 186ms/step - loss: 1.4306 - accuracy: 0.6979
Epoch 5/5
32/32 [==============================] - 8s 244ms/step - loss: 0.8900 - accuracy: 0.7693
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 3)                 6147      
                      

In [46]:
# test
test_loss, test_acc = model.evaluate(X_test, y_test)

7/7 [==============================] - 2s 129ms/step - loss: 2.7746 - accuracy: 0.4769
